In [1]:
import pandas as pd
import numpy as np

import wget
import zipfile
import os

if not os.path.isfile("rawinventor.tsv"):
    wget.download("https://s3.amazonaws.com/data.patentsview.org/download/rawinventor.tsv.zip")
    with zipfile.ZipFile("rawinventor.tsv.zip", 'r') as zip_ref:
        zip_ref.extractall(".")
    os.remove("rawinventor.tsv.zip")

rawinventor = pd.read_csv("rawinventor.tsv", sep="\t")
rawinventor["mention-id"] = "US" + rawinventor.patent_id.astype(str) + "-" + rawinventor.sequence.astype(str)
disambiguation = rawinventor.set_index("mention-id")["inventor_id"]

In [44]:
benchmark = pd.read_csv("inventor_benchmark_binette_20.csv", sep="\t").fillna("").iloc[0:20, :]

In [57]:
def lambd(x):
    cluster = disambiguation[disambiguation == x.inventor_id].index.values
    to_add = np.setdiff1d(x["add"].split(", "), [""])
    cluster = np.append(cluster, to_add)
    to_remove = np.setdiff1d(x["remove"].split(", "), [""])
    if len(to_remove) > 0:
        assert all(mention in cluster for mention in to_remove)
        cluster =  np.setdiff1d(cluster, to_remove)
    return cluster

true_clusters = benchmark.apply(lambd, axis=1)

In [74]:
reference = pd.concat({"inventor_id":benchmark.inventor_id, "mention-id":true_clusters}, axis=1).explode("mention-id").set_index("mention-id")["inventor_id"]

In [75]:
from pv_evaluation.estimators import pairwise_precision_estimator

pairwise_precision_estimator(disambiguation, reference, sampling_type="cluster_block", weights="cluster_size")

0.9759856470977679